In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mlflow-models-hammad/1', creation_time=1736592501932, experiment_id='1', last_update_time=1736592501932, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('../../01-intro/data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../../01-intro/data/green_tripdata_2024-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val) ** 1/2
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 14.797471227066088


2025/01/11 10:48:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run sedate-bug-612 at: http://127.0.0.1:5000/#/experiments/1/runs/4a0361088db0437e9308066a3ed80a55
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [7]:
from mlflow.tracking import MlflowClient


In [8]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '4a0361088db0437e9308066a3ed80a55'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
client.dow

In [9]:
path = client.download_artifacts(run_id=RUN_ID, path='model.bin')

MlflowException: The following failures occurred while downloading one or more artifacts from s3://mlflow-models-hammad/1/4a0361088db0437e9308066a3ed80a55/artifacts:
##### File model.bin #####
An error occurred (404) when calling the HeadObject operation: Not Found

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()